In [1]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
from scipy import spatial
from scipy.sparse.linalg import eigs
import matplotlib.pyplot as plt

git_path = '/Users/optas/Documents/Git_Repos/'
sys.path.insert(0, git_path)

from tf_lab.fundamentals import *
%matplotlib inline

In [9]:
from tensorflow.models.image.cifar10 import cifar10
cifar10.FLAGS.data_dir = '/Users/optas/DATA/Images/CIFAR_10/'
cifar10.maybe_download_and_extract()
NUM_CLASSES = 10

SyntaxError: invalid syntax (<ipython-input-9-67659e4d7d8d>, line 6)

In [7]:
pad = 'SAME'
stddev = 5e-2
    
def inference_network(in_signal):
    layer = convolutional_layer(in_signal, n_filters=64, filter_size=[5,5], stride=1, padding=pad, stddev=stddev, name='conv1')
    layer = max_pool(relu(layer), ksize=[3,3], stride=[2,2], name='max_pool_1')
    layer = convolutional_layer(in_signal, n_filters=64, filter_size=[5,5], stride=1, padding=pad, stddev=stddev, name='conv2')
    layer = max_pool(relu(layer), ksize=[3,3], stride=[2,2], name='max_pool_2')
    layer = fully_connected_layer(layer, 384, stddev=0.04, wd=0.004, init_bias=0.1, name='fc_1')
    layer = fully_connected_layer(layer, 192, stddev=0.04, wd=0.004, init_bias=0.1, name='fc_2')
    return fully_connected_layer(layer, NUM_CLASSES, stddev=1/102.0, init_bias=0.0, name='fc_3')

In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import os.path
import time

from six.moves import xrange

FLAGS = cifar10.FLAGS
max_steps = 1000000
batch_size = 128
train_dir  ='/Users/optas/DATA/Neural_Nets/Train_Log/'

with tf.Graph().as_default():
    global_step = tf.Variable(0, trainable=False)

    # Get images and labels for CIFAR-10.
    images, labels = cifar10.distorted_inputs()

    # Build a Graph that computes the logits predictions from the
    # inference model.
    logits = inference_network(images)
    loss = cifar10.loss(logits, labels)

    # Build a Graph that trains the model with one batch of examples and
    # updates the model parameters.
    train_op = cifar10.train(loss, global_step)

    # Create a saver.
    saver = tf.train.Saver(tf.all_variables())
    
    init = tf.initialize_all_variables()
    # Start running operations on the Graph.
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
    sess.run(init)

    # Start the queue runners.
    tf.train.start_queue_runners(sess=sess)
    
    for step in xrange(max_steps):
        start_time = time.time()
        _, loss_value = sess.run([train_op, loss])
        duration = time.time() - start_time

        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

        if step % 10 == 0:
            num_examples_per_step = batch_size
            examples_per_sec = num_examples_per_step / duration
            sec_per_batch = float(duration)

            format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f sec/batch)')
            print (format_str % (datetime.now(), step, loss_value, examples_per_sec, sec_per_batch))

#         if step % 100 == 0:
#             summary_str = sess.run(summary_op)
#             summary_writer.add_summary(summary_str, step)

        # Save the model checkpoint periodically.
#         if step % 1000 == 0 or (step + 1) == max_steps:
#             checkpoint_path = os.path.join(train_dir, 'model.ckpt')
#             saver.save(sess, checkpoint_path, global_step=step)



Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
2016-12-31 21:42:09.115782: step 0, loss = 11.26 (14.7 examples/sec; 8.686 sec/batch)
2016-12-31 21:42:10.883888: step 10, loss = 10.97 (729.5 examples/sec; 0.175 sec/batch)
2016-12-31 21:42:12.641867: step 20, loss = 10.78 (718.9 examples/sec; 0.178 sec/batch)
2016-12-31 21:42:14.391341: step 30, loss = 10.74 (749.7 examples/sec; 0.171 sec/batch)
2016-12-31 21:42:16.120942: step 40, loss = 10.40 (736.5 examples/sec; 0.174 sec/batch)
2016-12-31 21:42:17.855835: step 50, loss = 10.52 (730.7 examples/sec; 0.175 sec/batch)
2016-12-31 21:42:19.573557: step 60, loss = 10.71 (747.3 examples/sec; 0.171 sec/batch)
2016-12-31 21:42:21.334009: step 70, loss = 10.30 (701.0 examples/sec; 0.183 sec/batch)
2016-12-31 21:42:23.109181: step 80, loss = 10.33 (731.6 examples/sec; 0.175 sec/batch)
2016-12-31 21:42:24.860706: step 90, loss = 10.29 (740.5 examples/sec; 0.173 sec/batch)
2016-12-31 21:42:26.578060: 

KeyboardInterrupt: 

In [6]:
import math

checkpoint_file = '/Users/optas/DATA/Neural_Nets/Train_Log/CIFAR10/TF_LAB_version/model.ckpt-72000'
num_examples = 10000
batch_size = 128

with tf.Graph().as_default():
    images, labels = cifar10.inputs(eval_data='test')    
    logits = inference_network(images)
    propbs = tf.nn.softmax(logits)    
    top_k_op = tf.nn.in_top_k(propbs, labels, 1)   
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())
    tr_vars = tf.trainable_variables()
    saver = tf.train.Saver(tr_vars)
    saver.restore(sess, checkpoint_file)
    # Start the queue runners.
    tf.train.start_queue_runners(sess=sess)
    
#     global_step = checkpoint_file.split('/')[-1].split('-')[-1]        
    predictions = sess.run([top_k_op])
        
    num_iter = int(math.ceil(num_examples / batch_size))
    true_count = 0  # Counts the number of correct predictions.
    total_sample_count = num_iter * batch_size
    step = 0
    while step < num_iter:
        predictions = sess.run([top_k_op])
        true_count += np.sum(predictions)
        step += 1

#       # Compute precision @ 1.
    precision = true_count / float(total_sample_count)
    print('precision @ 1 = %.3f' % (precision))

6331
precision @ 1 = 0.634


array([ 0.34447801, -3.29785943,  0.05256522,  5.14447784, -0.26849735,
        0.82257384,  2.97710323, -2.84878302,  1.01652539, -4.02707672], dtype=float32)